In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%cd ..

/Users/kristophermiltiadou/Documents/UniWork/Cambridge/Thesis/CODE/iREC


In [24]:
import torch
import torch.distributions as dist
import math
import matplotlib.pyplot as plt

from rec.beamsearch.distributions.CodingSampler import CodingSampler
from rec.beamsearch.distributions.VariationalPosterior import VariationalPosterior
from rec.beamsearch.distributions.EmpiricalMixturePosterior import EmpiricalMixturePosterior
from rec.beamsearch.samplers.GreedySampling import GreedySampler
from rec.beamsearch.Coders.Encoder import Encoder as Empirical_Encoder
from rec.beamsearch.Coders.Encoder_Variational import Encoder as Variational_Encoder
from models.BayesianLinRegressor import BayesLinRegressor
from rec.utils import kl_estimate_with_mc, compute_variational_posterior, plot_samples_in_2d, plot_running_sum_2d, plot_pairs_of_samples

In [35]:
def encode_sample(target, method='empirical', omega=8, epsilon=0.,
                  n_empirical_samples=10, seed=0, beamwidth=1):
    
    if method == 'empirical':
        encoder = Empirical_Encoder(target,
                                    seed,
                                    CodingSampler,
                                    GreedySampler,
                                    EmpiricalMixturePosterior,
                                    omega,
                                    n_empirical_samples,
                                    epsilon=epsilon,
                                    beamwidth=beamwidth
                                    )
    else:
        target = compute_variational_posterior(target)
        encoder = Variational_Encoder(target,
                                      seed,
                                      CodingSampler,
                                      GreedySampler,
                                      VariationalPosterior,
                                      omega,
                                      epsilon=epsilon,
                                      beamwidth=beamwidth
                                      )

    return encoder, *encoder.run_encoder()

In [36]:
def compute_metrics(blr, z, encoder):
    target = blr.weight_posterior
    best_sample_idx = torch.argmax(target.log_prob(z))
    best_sample = z[best_sample_idx]
    MSE = blr.measure_performance(best_sample, kind='MSE')
    MAE = blr.measure_performance(best_sample, kind='MAE')
    mahalobonis_dist = torch.sqrt((target.mean - best_sample).T@target.covariance_matrix @(target.mean - best_sample))
    log_q_over_p = target.log_prob(best_sample) - encoder.auxiliary_posterior.coding_sampler.log_prob(best_sample)
    
    return MSE, MAE, mahalobonis_dist, log_q_over_p

In [37]:
def performance_drop_off(target, MSE, MAE):
    MSE_dropoff = (MSE - blr.measure_performance(target.mean, kind='MSE')) / blr.measure_performance(target.mean, kind='MSE') * 100
    MAE_dropoff = (MAE - blr.measure_performance(target.mean, kind='MAE')) / blr.measure_performance(target.mean, kind='MAE') * 100
    
    return MSE_dropoff, MAE_dropoff

# 20 Dimension Problem

In [38]:
torch.set_default_tensor_type(torch.DoubleTensor)
initial_seed_target = 69
blr = BayesLinRegressor(prior_mean=torch.zeros(20),
                        prior_alpha=1,
                        signal_std=1,
                        num_targets=10000,
                        seed=initial_seed_target)
blr.sample_feature_inputs()
blr.sample_regression_targets()
blr.posterior_update()
target = blr.weight_posterior

# Beamsearch comparison

In [ ]:
beamwidths = [1, 10, 15, 20]
random_seeds = [1e2]

empirical_scheme_full_results = []
empirical_scheme_dropoffs = []
variational_scheme_full_results = []
variational_scheme_dropoffs = []

for beam in beamwidths:
    emp_MSE = 0
    emp_MAE = 0
    emp_mah_dist = 0
    emp_log_q_p = 0
    var_MSE = 0
    var_MAE = 0
    var_mah_dist = 0
    var_log_q_p = 0
    for seed in random_seeds:
        emp_enc, emp_z, _ = encode_sample(target, method='empirical', seed=seed, beamwidth=beam, epsilon=0.)
        var_enc, var_z, _ = encode_sample(target, method='Variational', seed=seed, beamwidth=beam, epsilon=0.)
        
        emp_stats = compute_metrics(blr, emp_z, emp_enc)
        emp_MSE += emp_stats[0]
        emp_MAE += emp_stats[1]
        emp_mah_dist += emp_stats[2]
        emp_log_q_p += emp_stats[3]
        
        var_stats = compute_metrics(blr, var_z, var_enc)
        var_MSE += var_stats[0]
        var_MAE += var_stats[1]
        var_mah_dist += var_stats[2]
        var_log_q_p += var_stats[3]
    empirical_scheme_full_results.append([emp_MSE / len(random_seeds),
                                         emp_MAE / len(random_seeds),
                                         emp_mah_dist / len(random_seeds),
                                         emp_log_q_p / len(random_seeds)])
    empirical_scheme_dropoffs.append([performance_drop_off(target, 
                                                          emp_MSE / len(random_seeds),
                                                          emp_MAE / len(random_seeds))])
    variational_scheme_full_results.append([var_MSE / len(random_seeds),
                                           var_MAE / len(random_seeds),
                                           var_mah_dist / len(random_seeds),
                                           var_log_q_p / len(random_seeds)])
    variational_scheme_dropoffs.append([performance_drop_off(target, 
                                                            var_MSE / len(random_seeds),
                                                            var_MAE / len(random_seeds))])

In [111]:
empirical_scheme_dropoffs

[[(tensor(1.2398), tensor(0.6422))],
 [(tensor(1.5876), tensor(0.7093))],
 [(tensor(1.5876), tensor(0.7093))],
 [(tensor(1.4532), tensor(0.6497))]]

In [112]:
variational_scheme_dropoffs

[[(tensor(1.2398), tensor(0.6422))],
 [(tensor(1.1680), tensor(0.6251))],
 [(tensor(1.1680), tensor(0.6251))],
 [(tensor(1.1680), tensor(0.6251))]]

In [113]:
emp_log_q_p = [x[-1] for x in empirical_scheme_full_results]
var_log_q_p = [x[-1] for x in variational_scheme_full_results]

In [114]:
var_log_q_p

[tensor(41.9756), tensor(45.5716), tensor(45.5716), tensor(45.5716)]

In [115]:
emp_log_q_p

[tensor(41.9756), tensor(24.5593), tensor(24.5593), tensor(31.2903)]

# Epsilon Comparison

In [ ]:
epsilons = [0., 0.05, 0.1, 0.2, 0.3]
random_seeds = [1e2]

empirical_scheme_full_results_epsilon = []
empirical_scheme_dropoffs_epsilon = []
variational_scheme_full_results_epsilon = []
variational_scheme_dropoffs_epsilon = []

for eps in epsilons:
    emp_MSE = 0
    emp_MAE = 0
    emp_mah_dist = 0
    emp_log_q_p = 0
    var_MSE = 0
    var_MAE = 0
    var_mah_dist = 0
    var_log_q_p = 0
    for seed in random_seeds:
        emp_enc, emp_z, _ = encode_sample(target, method='empirical', seed=seed, beamwidth=10, epsilon=eps)
        var_enc, var_z, _ = encode_sample(target, method='Variational', seed=seed, beamwidth=10, epsilon=eps)
        
        emp_stats = compute_metrics(blr, emp_z, emp_enc)
        emp_MSE += emp_stats[0]
        emp_MAE += emp_stats[1]
        emp_mah_dist += emp_stats[2]
        emp_log_q_p += emp_stats[3]
        
        var_stats = compute_metrics(blr, var_z, var_enc)
        var_MSE += var_stats[0]
        var_MAE += var_stats[1]
        var_mah_dist += var_stats[2]
        var_log_q_p += var_stats[3]
    empirical_scheme_full_results_epsilon.append([emp_MSE / len(random_seeds),
                                         emp_MAE / len(random_seeds),
                                         emp_mah_dist / len(random_seeds),
                                         emp_log_q_p / len(random_seeds)])
    empirical_scheme_dropoffs_epsilon.append([performance_drop_off(target, 
                                                          emp_MSE / len(random_seeds),
                                                          emp_MAE / len(random_seeds))])
    variational_scheme_full_results_epsilon.append([var_MSE / len(random_seeds),
                                           var_MAE / len(random_seeds),
                                           var_mah_dist / len(random_seeds),
                                           var_log_q_p / len(random_seeds)])
    variational_scheme_dropoffs_epsilon.append([performance_drop_off(target, 
                                                            var_MSE / len(random_seeds),
                                                            var_MAE / len(random_seeds))])

 42%|████▏     | 5/12 [04:59<08:01, 68.75s/it]

In [107]:
empirical_scheme_dropoffs_epsilon

[[(tensor(1.5876), tensor(0.7093))],
 [(tensor(1.6215), tensor(0.7712))],
 [(tensor(1.1446), tensor(0.5927))],
 [(tensor(0.5409), tensor(0.3139))],
 [(tensor(0.4386), tensor(0.2412))]]

In [109]:
variational_scheme_dropoffs_epsilon

[[(tensor(1.1680), tensor(0.6251))],
 [(tensor(1.6215), tensor(0.7712))],
 [(tensor(1.1446), tensor(0.5927))],
 [(tensor(0.5409), tensor(0.3139))],
 [(tensor(0.5616), tensor(0.2828))]]